In [1]:
import pandas as pd
import openai
from utils import *
import constants
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
openai.api_key = constants.OPENAI_API_KEY

### Query S2 paper index

In [3]:
query = "How does iron supplementation affect anemia?"

# json to pandas dataframe
search_results = search(preprocess_query(query))

if search_results['total']==0:
    print('No results found - Try another query')
else:
    df = pd.DataFrame(search_results['data']).dropna()

In [4]:
df.head()

,paperId,title,abstract
0,d17242c18f0bf942f258db2fb163933e3ea1a8f0,Iron Fortification and Supplementation: Fighti...,ABSTRACT The significant worldwide increase in...
2,c910b35eb94711fb0f17075f88e132f8cf0bf85a,Iron-containing micronutrient supplementation ...,Universal prenatal daily iron-folic acid (IFA)...
3,e056f91c0ba1bc3214e0f89aaf375cafc95f3da3,Effectivity of Iron Supplementation With and W...,Anemia can be caused by a decrease in hemoglob...
4,2cbf181eaa9bf0ab7bc4d5426da4cabf6cea53bf,Effects of iron deficiency and iron supplement...,Iron deficiency is the most prevalent human mi...
6,459828144e59ace72493adae1068d460d0989c46,Benefits and Risks of Early Life Iron Suppleme...,Infants are frequently supplemented with iron ...


### Re-ranking using SPECTER

In [5]:
# merge columns title and abstract into a string separated by tokenizer.sep_token and store it in a list

df['title_abs'] = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for d in df.to_dict('records')]
df['n_tokens'] = df.title_abs.apply(lambda x: len(tokenizer.encode(x)))
df['title_abs'][2]

'Iron-containing micronutrient supplementation of Chinese women with no or mild anemia during pregnancy improved iron status but did not affect perinatal anemia.[SEP]Universal prenatal daily iron-folic acid (IFA) and multiple micronutrient (MM) supplements are recommended to reduce the risk of low birth weight, maternal anemia, and iron deficiency (ID) during pregnancy, but the evidence of their effect on iron status among women with mild or no anemia is limited. The aim of this study was to describe the iron status [serum ferritin (SF), serum soluble transferrin receptor (sTfR), and body iron (BI)] before and after micronutrient supplementation during pregnancy. We examined 834 pregnant women with hemoglobin > 100 g/L at enrollment before 20 wk of gestation and with iron measurement data from a subset of a randomized, double-blind trial in China. Women were randomly assigned to take daily 400 μg of folic acid (FA) (control), FA plus 30 mg of iron, or FA, iron, plus 13 additional MMs p

In [6]:
# get embeddings for each document and query

doc_embeddings = get_specter_embeddings(list(df['title_abs']))
query_embeddings = get_specter_embeddings(query)

df['specter_embeddings'] = list(doc_embeddings)
df.head()

,paperId,title,abstract,title_abs,n_tokens,specter_embeddings
0,d17242c18f0bf942f258db2fb163933e3ea1a8f0,Iron Fortification and Supplementation: Fighti...,ABSTRACT The significant worldwide increase in...,Iron Fortification and Supplementation: Fighti...,195,"[-0.2920975, -0.36597273, 0.23700278, 0.480033..."
2,c910b35eb94711fb0f17075f88e132f8cf0bf85a,Iron-containing micronutrient supplementation ...,Universal prenatal daily iron-folic acid (IFA)...,Iron-containing micronutrient supplementation ...,470,"[-0.54441655, -0.5294113, 0.43898264, 0.709588..."
3,e056f91c0ba1bc3214e0f89aaf375cafc95f3da3,Effectivity of Iron Supplementation With and W...,Anemia can be caused by a decrease in hemoglob...,Effectivity of Iron Supplementation With and W...,353,"[-0.39489794, -0.46253267, 0.5966403, 0.662496..."
4,2cbf181eaa9bf0ab7bc4d5426da4cabf6cea53bf,Effects of iron deficiency and iron supplement...,Iron deficiency is the most prevalent human mi...,Effects of iron deficiency and iron supplement...,397,"[-0.37867287, -0.52501506, 0.30210197, 0.62727..."
6,459828144e59ace72493adae1068d460d0989c46,Benefits and Risks of Early Life Iron Suppleme...,Infants are frequently supplemented with iron ...,Benefits and Risks of Early Life Iron Suppleme...,130,"[-0.4462546, -0.43935454, 0.33284977, 0.610091..."


In [7]:
# find the cosine similarity between the query and the documents
df['similarity'] = cosine_similarity(query_embeddings, doc_embeddings).flatten()

# sort the dataframe by similarity
df.sort_values(by='similarity', ascending=False, inplace=True)
df.head()

,paperId,title,abstract,title_abs,n_tokens,specter_embeddings,similarity
11,a6d2264066edf46f03f1c7df46b0e8079f8e0b63,Efficacy of daily versus intermittent administ...,Iron deficiency anemia is one of the main caus...,Efficacy of daily versus intermittent administ...,316,"[-0.50734, -0.4384992, 0.5872712, 0.7197657, 0...",0.903400
3,e056f91c0ba1bc3214e0f89aaf375cafc95f3da3,Effectivity of Iron Supplementation With and W...,Anemia can be caused by a decrease in hemoglob...,Effectivity of Iron Supplementation With and W...,353,"[-0.39489794, -0.46253267, 0.5966403, 0.662496...",0.897092
13,f312120e9e0fa93842dadf5cc55ee367e4d22615,The effect of iron supplementation on birth ou...,\n Background: Although Iron supplementation i...,The effect of iron supplementation on birth ou...,391,"[-0.47750962, -0.45246515, 0.48677686, 0.73421...",0.890646
6,459828144e59ace72493adae1068d460d0989c46,Benefits and Risks of Early Life Iron Suppleme...,Infants are frequently supplemented with iron ...,Benefits and Risks of Early Life Iron Suppleme...,130,"[-0.4462546, -0.43935454, 0.33284977, 0.610091...",0.883864
2,c910b35eb94711fb0f17075f88e132f8cf0bf85a,Iron-containing micronutrient supplementation ...,Universal prenatal daily iron-folic acid (IFA)...,Iron-containing micronutrient supplementation ...,470,"[-0.54441655, -0.5294113, 0.43898264, 0.709588...",0.881683


## Get answer from GPT-3

In [8]:
answer_question(df, question=query, debug=False)

'Iron supplementation can reduce anemia in pregnant women with mild or no anemia, but it can also increase the risk of neonatal jaundice. Iron supplementation can also improve iron stores and decrease anemia in non-pregnant women, but it can also increase the risk of diarrhea. Good adherence and initiation of supplementation before conception are needed to reduce anemia during early pregnancy.'